In [1]:
import pandas as pd
import numpy as np
import bllipparser
import json
import os
from nltk.corpus import gutenberg, reuters
from nltk.tree import Tree

In [2]:
rrp = bllipparser.RerankingParser.fetch_and_load('WSJ+Gigaword-v2', verbose=True)

Model directory: /Users/kesslej/.local/share/bllipparser/WSJ+Gigaword-v2
Model directory already exists, not reinstalling


In [3]:
nltk.download('gutenberg')
nltk.download('reuters')

NameError: name 'nltk' is not defined

In [129]:
'''
def get_sentences_from_dataset(dataset):
    data = []
    for fileid in dataset.fileids()[:100]:
        data.append({'fileid':fileid, 'raw': dataset.raw(fileid)})
    return pd.DataFrame(data)

def parse_with_corenlp(df, intermediate_file_name = 'raw.txt'):
    df['raw'].to_csv(intermediate_file_name, index=Fa)
    os.system(('java -cp "{}/*" '
               '-Xmx500m edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators '
               'tokenize,ssplit,pos,parse -file {} -outputFormat json'
              ).format(CORENLP_PATH, intermediate_file_name))
    
def get_treelist(intermediate_file_name = 'raw.txt'):
    trees = []
    for l in list(open(intermediate_file_name + '.json')):
        l = l.strip()
        if l.startswith('"parse":'):
            trees.append(' '.join(json.loads('{'+l[:-1]+'}')['parse'].split()))
    return pd.DataFrame({'tree': trees})
'''

In [132]:
#%time
#df = get_sentences_from_dataset(reuters)
#parse_with_corenlp(df)
get_treelist().to_csv('trees.csv', index=False)

In [163]:
def iter_top_nps(tree):
    if isinstance(tree, Tree):
        if tree.label() == 'NP':
            yield tree
        else:
            for subtree in tree:
                for tree in iter_top_nps(subtree):
                    yield tree
                    
def get_numbers_from_tree(tree):
    for subtree in tree:
        if subtree.label() == 'NP':
            #print('examining', ' '.join(subtree.leaves()))
            labels = [subtree.label() for subtree in tree if subtree.label() not in (',', 'CC')]
            #print('labels', labels, len(set(labels)) == 1 and labels[0] == 'NP')
            if len(set(labels)) == 1 and labels[0] == 'NP':
                for subtree in tree:
                    if subtree.label() == 'NP':
                        for subsubtree in iter_top_nps(subtree):
                            yield get_number_from_np(subtree)
            else:
                #print('returning', tree)
                yield get_number_from_np(subtree)

def get_number_from_np(tree):
    pos_list = [pos for orth, pos in tree.pos()]
    if pos_list[-1][-1] == 'S' or 'CC' in pos_list:
        return 'P'
    return 'S'

def which_is_more_ambiguous(numbers):
    if numbers[-3:] == ['S', 'S', 'S']: return 'Oxford' # my mother, Jill, and Sam
    if numbers[-3:] == ['S', 'S', 'P']: return 'Oxford' # my mother, Jill, and the Smiths
    if numbers[-3:] == ['S', 'P', 'S']: return 'Neither' # my mother, the Smiths, and Sam
    if numbers[-3:] == ['S', 'P', 'P']: return 'Neither' # my mother, the Smiths, and the Jonese
    if numbers[-3:] == ['P', 'S', 'S']: return 'Lack of Oxford' # my parents, Jill and Sam
    if numbers[-3:] == ['P', 'P', 'S']: return 'Lack of Oxford' # my family, the Smiths and Sam
    if numbers[-3:] == ['P', 'S', 'P']: return 'Lack of Oxford' # my family, Sam and the Smiths
    if numbers[-3:] == ['P', 'P', 'P']: return 'Lack of Oxford' # my family, the Smiths and the Joneses
        
def analyze_tree(tree):
    for np in iter_top_nps(tree):
        numbers = list(get_numbers_from_tree(np))
        if len(numbers) > 2:
            yield [np, numbers, which_is_more_ambiguous(numbers)]

In [178]:
tree = Tree.fromstring('''(NP
  (NP (NNP Tom) (NNP Murtha))
  (, ,)
  (NP
    (NP (DT a) (NN stock) (NN analyst))
    (PP
      (IN at)
      (NP
        (NP (DT the) (NNP Tokyo) (NN office))
        (PP (IN of) (NP (NN broker)))))))''')
[[t.label(), [e.label() for e in t if type(e) == Tree]] for t in tree]

[['NP', ['NNP', 'NNP']], [',', []], ['NP', ['NP', 'PP']]]

In [183]:
for tree in get_treelist()['tree'].iloc[:40]:
    for x in analyze_tree(Tree.fromstring(tree)):
        print(' '.join(x[0].leaves()))
        print(x[1:])

0 , `` ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT Mounting trade friction between the U.S. And Japan has raised fears among many of Asia 's exporting nations that the row could inflict far-reaching economic damage , businessmen and officials
[['S', 'P', 'S', 'P'], 'Lack of Oxford']
firm Matsushita Electric Industrial Co Ltd < MC.T >
[['S', 'S', 'S', 'S'], 'Oxford']
Tom Murtha , a stock analyst at the Tokyo office of broker
[['S', 'S', 'S', 'S'], 'Oxford']
the world 's largest
[['P', 'P', 'P', 'P'], 'Lack of Oxford']
Paul Sheen , chairman of textile exporters
[['S', 'P', 'S', 'P'], 'Lack of Oxford']
the U.S. , Up from 4.9 billion dlrs in 1985
[['S', 'S', 'S', 'S'], 'Oxford']
coal and beef , Australia 's two largest exports
[['P', 'P', 'P', 'P'], 'Lack of Oxford']
Deputy U.S. Trade Representative Michael Smith and Makoto Kuroda , Japan 's deputy minister of International Trade and Industry -LRB- MITI -RRB- ,
[['P', 'P', 'P', 'P'], 'Lack of Oxford']
improved technology in storage an